In [ ]:
# System imports
import os
import sys
import functools as ft

# 3rd-party imports
import numpy as np
import keras
from ipyparallel.datapub import publish_data
import ipyparallel as ipp

# Local imports
import ipcluster_magics
from mnist import load_data
from updatingplot import UpdatingPlot
from mlextras import build_and_train
from paramspan import ParamSpanWidget

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Start IPyParallel Cluster
import ipcluster_magics

In [ ]:
%ipcluster -m tensorflow/intel-1.8.0-py36 -N 2

In [ ]:
# Connect to IPP controller
c = ipp.Client()
lv = c.load_balanced_view()
dv = c.direct_view()
print(c.ids)

In [ ]:
# Make sure tensorflow modules are available remotely
def set_path(path):
    global sys
    import sys
    sys.path = path
dv.apply(set_path, sys.path)
workdir = os.path.expanduser('~/cori-intml-examples')
%px cd "$workdir"

In [ ]:
x_train, y_train, x_test, y_test = load_data()

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

In [ ]:
# Training config
# Hold these parameters constant
batch_size = 128
n_epochs = 16
valid_frac = 0.17
checkpoint_dir = '/global/cscratch1/sd/oevans/cori-interactive-dl/mnist-hpo'
checkpoint_file = os.path.join(checkpoint_dir, 'model_single.h5')
n_samples = 1000

const = dict(
    checkpoint_file=checkpoint_file, verbose=0,
    x_train=x_train[:n_samples], 
    y_train=y_train[:n_samples],
    valid_frac=valid_frac,
    batch_size=batch_size, n_epochs=n_epochs,
)

In [ ]:
run_training = ft.partial(
    build_and_train,
    **const
)

plot_metrics = ft.partial(
    UpdatingPlot, 
    y=['loss', 'acc'],
    xlim=[0, n_epochs],
    xlabel='epochs',
    ylabel='training metrics'
)

# Single Parameter Set

In [ ]:
h1 = 16
h2 = 16
h3 = 32
dropout=0.5
optimizer = 'Adadelta'

n_samples = 1000

opts = dict(
    h1=h1, h2=h2, h3=h3,
    dropout=dropout, 
    optimizer=optimizer
)

# Several Parameter Sets

In [ ]:
n_hpo_trials = 8
grid_h1 = np.random.choice([4, 8, 16, 32, 64], size=n_hpo_trials)
grid_h2 = np.random.choice([4, 8, 16, 32, 64], size=n_hpo_trials)
grid_h3 = np.random.choice([8, 16, 32, 64, 128], size=n_hpo_trials)
grid_dropout = np.random.rand(n_hpo_trials)
grid_optimizer = np.random.choice(['Adadelta', 'Adam', 'Nadam'], size=n_hpo_trials)

In [ ]:
psw = ParamSpanWidget(run_training, plot_metrics)

psw.set_params(
    h1=grid_h1,
    h2=grid_h2,
    h3=grid_h3,
    dropout=grid_dropout,
    optimizer=grid_optimizer
)

psw.submit_computations()

psw